In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("HW_3_robot_arm.ipynb")

# Robot arm

Part 1: Forward kinematics

Part 2: Optimization

Slides: https://docs.google.com/presentation/d/17aiTBmPZidR6op7TvqYRzYatuc_NETYA1BhgpSHQ-FM/edit?usp=sharing

In [ ]:
# The usual imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fmin

In [ ]:
# TODO If you wrote your functions in the Lab JN copy them back to arm_routines.py at this point. Otherwise, nothing will work.

In [ ]:
# matrix_rouintes.py functions
import matrix_routines as mt

# These are the routines in arm_routines.py you'll need, but you don't need to change/edit them
from arm_routines import create_arm_component, create_arm_geometry, plot_object, plot_complete_arm

In [ ]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

# Doing forward kinematics

TODO: Edit the **set_matrices_all_components** function. This function has the same basic flow as **set_rotation_matrices_all_components** in the lab. This time, however, you're going to set Matrix_pose to be the matrix that moves the commponent to its final location.


In [ ]:
# Create the arm geometry (done in Lab 5)
base_size_param = (0.5, 1.0)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)


# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_geometry_fk = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

In [ ]:
# EXAMPLE CODE
# Getting out where the base of the matrix is and what rotation to use.

# This is the matrix that transforms the base
#  The first item in the arm geometry list is the base
matrix_base = arm_geometry_fk[0]
matrix_wedge_no_transform = create_arm_component(name="Base No transform", pts=matrix_base["Pts"], color="grey")

# Draw the wedge in it's original location, with the point we're interested in (and direction)
pt_on_top = np.array([0, 1, 1]).transpose()
vec_on_top = np.array([0, 1, 0]).transpose()
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
axs[0].set_title("Wedge with pt")
plot_object(axs[0], matrix_wedge_no_transform, b_do_pose_matrix=False)
axs[0].plot(pt_on_top[0], pt_on_top[1], 'Xk')
axs[0].arrow(x=pt_on_top[0], y=pt_on_top[1], dx=vec_on_top[0], dy=vec_on_top[1], color="red")

# Multiply the point and the vector by the matrix base's matrix
pt_on_top_moved = matrix_base["Matrix_shape"] @ pt_on_top
vec_on_top_moved = matrix_base["Matrix_shape"] @ vec_on_top
axs[1].set_title("Wedge with pt moved")
plot_object(axs[1], matrix_base, b_do_pose_matrix=False)
axs[1].plot(pt_on_top_moved[0], pt_on_top_moved[1], 'Xk')
axs[1].arrow(x=pt_on_top_moved[0], y=pt_on_top_moved[1], dx=vec_on_top_moved[0], dy=vec_on_top_moved[1], color="red")

# arctan2 gets the arc tangent of the y, x, and correctly handles the quadrants
angle_of_rotation = np.arctan2(vec_on_top_moved[1], vec_on_top_moved[0])

print(f"Angle rotated by {angle_of_rotation}")

In [ ]:
# Add one cell for each additional function you add
...


In [ ]:

def set_matrices_all_components(arm_list, angles_list):
    """ For each component, set the matrix that takes the link to its final location.
    In other words, set the "Matrix_pose" key in each dictionary in arm_link
    
    Base matrix (arm_list[0]): The matrix to multiply the base by (should be the identity)
    
    First link matrix (arm_list[1]): The matrix to multiply the first link by in order to attach it to the base, rotated by the
       given amount, pointed up

    Second link matrix (arm_list[2]): The matrix to multiply the second link by in order to attach it to the end of the first
       link, rotated by the given amount
       ...
    
    palm matrix (arm_list[-1][0]): The matrix to multiply the gripper by in order to attach it to the last link, rotated by the given amount
    finger matrices (arm_list[-1][1:2]): (optional) Rotate the fingers/open/close them
    
    @param arm_list - the list of the arm component dictionaries.
    @param angles_list - a list of angles, one for each link, one for the palm, and one for the fingers
    """

    # TODO: Set the "Matrix_pose" matrix for each arm component

    # The base link - it doesn't move, so this is the identity matrix. Setting here just to show the syntax of
    #   setting the Matrix_pose keye
    arm_list[0]["Matrix_pose"] = np.identity(3)

    # TODO:
    #  The first link matrix needs to rotate the iink, then translate it to the top of the base, pointing up
    #  Build the latter part of that matrix by figuring out where (0.0, 1.0) on the base wedge went in the world coordinate. (see Example code in JN)
    #    Remember that base_link is a dictionary; the matrix that shapes the base is in "Matrix_shape"
    #  Step 1: Get the matrix from the base link
    #  Step 2: Find the point in the world coordinate, which is (0.0,1.0) in the base wedge coordinate.
    #          (point_in_world = matrix @ point_in_local, Remember that the matrix is 3x3.)
    #  Step 3: Rotate first to point up, then translate (Point in step 2)

    ...

    for ang, link in zip(angles_list[0:-1], arm_list[1:-1]):
        link["Angle"] = ang
        link["Matrix_pose"] = np.identity(3)
        # TODO 
        # For the Lab: Set the matrix in the key "Matrix_pose" to be a rotation for that link
        # For the HWk: Keep a matrix that does all of the transforms up to that point (getting the link
        #  to the end of the last link and then rotating by that link) then append the rotation matrix for the link
        # See Homework slides
        ...

    # Now do the gripper (palm) - the fingers are optional
    # If you don't do the fingers, just use the same matrix for the palm AND the fingers
    gripper = arm_list[-1]
    angles_gripper = angles_list[-1]
    palm_obj = gripper[0]
    palm_obj["Angle"] = angles_gripper[0]
    palm_obj["Matrix_pose"] = np.identity(3)

    # TODO Set the matrix pose for the palm. This is exactly the same as setting the link pose
    ...

    # TODO (optional):
    #   Translate the base of the finger back to the origin, rotate it, then translate it back out
    #   Reminder: The middle of the finger can be found using mt.get_dx_dy_from_matrix
    #    Note: You want to move the base of the finger, NOT the middle, to the origin before you do the rotate
    for finger, finger_angle in zip(gripper[1:3], angles_gripper[1:]):
        finger["Angle"] = finger_angle
        ...



In [ ]:
## Check the combined link/gripper/finger rotations
# Several different angles to check your routines with 
# Pass the one you want to check into set_angles_of_arm_geometry below
#  Feel free to change these
angles_none = [0.0, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_link_0 = [np.pi/4, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_link_0_1 = [np.pi/4, -np.pi/4, 0.0, [0.0, 0.0, 0.0]]
angles_check_wrist = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, 0.0, 0.0]]
angles_check_fingers = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [0.0, np.pi/4.0, -np.pi/4.0]]

# Don't change this one
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]


In [ ]:
# Use this cell to visually check the results
# With angles_none it should point straight up
# The cells below will check each inidividual matrix 
set_matrices_all_components(arm_geometry_fk, angles_check)

fig, axs = plt.subplots(1, 1, figsize=(6, 6))
plot_complete_arm(axs, arm_geometry_fk)

In [ ]:
# Don't change this one - the cells below assume these are the angles
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]
set_matrices_all_components(arm_geometry_fk, angles_check)


In [ ]:
# Check the returned values
np.set_printoptions(precision=4, floatmode='fixed')  # Print out with 4 digits of precision

mat_check_base = np.identity(3)
print(arm_geometry_fk[0]["Matrix_pose"])
assert(np.all(np.isclose(arm_geometry_fk[0]["Matrix_pose"], mat_check_base, atol=0.01)))

In [ ]:
mat_check_link_1 = np.array([[ -1.0,  0.0,  0.0], \
                             [  0.0, -1.0,  0.5], \
                             [  0.0,  0.0,  1.0]])
                             
print(arm_geometry_fk[1]["Matrix_pose"])
assert(np.all(np.isclose(arm_geometry_fk[1]["Matrix_pose"], mat_check_link_1, atol=0.01)))

In [ ]:
mat_check_link_2 = np.array([[ -0.7071, -0.7071, -0.5], \
                             [  0.7071, -0.7071,  0.5], \
                             [  0.0,     0.0,  1.0]])
print(arm_geometry_fk[2]["Matrix_pose"])
assert(np.isclose(np.sum(mat_check_link_2 - arm_geometry_fk[2]["Matrix_pose"]), 0.0, atol=0.1))

In [ ]:
mat_check_link_3 = np.array([[ 1.0, 0.0, -0.71213], \
                             [ 0.0, 1.0,  0.71213], \
                             [ 0.0, 0.0,  1.0]])
print(arm_geometry_fk[3]["Matrix_pose"])
assert(np.all(np.isclose(arm_geometry_fk[3]["Matrix_pose"], mat_check_link_3, atol=0.01)))

In [ ]:
mat_check_wrist = np.array([[ 0.5, -0.8660,  -0.5121], \
                            [ 0.8660,  0.5,   0.7121], \
                            [ 0.0,  0.0,  1.0]])
print(arm_geometry_fk[-1][0]["Matrix_pose"])
assert(np.all(np.isclose(arm_geometry_fk[-1][0]["Matrix_pose"], mat_check_wrist, atol=0.01)))

In [ ]:
grader.check("forward_kinematrix")

# Gripper location

TODO: edit **get_gripper_location** to return the x,y location of the gripper.

A reminder that the grasp location is offset from the palm by a bit (["grasp"] key in palm object)

In [ ]:
def get_gripper_location(arm_list):
    """ Get the gripper grasp location (between the fingers) given the arm
    Assumes the Matrix_pose keys have been set already
    Use the "Grasp" key in the gripper to figure out how far to offset the  point from the base of the gripper
    @param arm_list
    @return x,y as a tuple - the location of the "grasp" point in the gripper
    """
    gripper = arm_list[-1]
    grasp_dist = gripper[0]["Grasp"]  # The distance out along the x axis that we'll call the "grasp" point

    # TODO:
    # Step 1: Get the matrix for the palm
    # Step 2: Multiply this matrix by [grasp_dist, 0] to get the location in world coordinates
    ...
    # Format for returning a tuple
    return (0, 0)


In [ ]:
# Check the gripper function
# As in the previous problem, you can use the "simpler" angles to check your function
angles_check_gripper = [np.pi/3, -np.pi/6, 3.0 * np.pi/6, [-np.pi/4, np.pi/4.0, -np.pi/4.0]]

# Making another copy of the arm geometry to use in these problems
arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Actually set the matrices
set_matrices_all_components(arm_geometry, angles_check_gripper)

# Check the trasp location is correct (there is plotting code in the next cell)
grasp_loc = get_gripper_location(arm_geometry)
assert(np.isclose(grasp_loc[0], -0.8106, atol=0.01) and np.isclose(grasp_loc[1], 0.92437, atol=0.01))


In [ ]:

# Now actually plot - the grasper grip location should show up as a green cross
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
plot_complete_arm(axs, arm_geometry)

axs.plot(grasp_loc[0], grasp_loc[1], '+g', label="Grasp location")



In [ ]:
# Do NOT change these or the autograder tests won't work
angles_check_gripper = [np.pi/3, -np.pi/6, 3.0 * np.pi/6, [-np.pi/4, np.pi/4.0, -np.pi/4.0]]

# Actually set the matrices
set_matrices_all_components(arm_geometry, angles_check_gripper)

grasp_loc = get_gripper_location(arm_geometry)

In [ ]:
grader.check("gripper_loc")

# FMin optimization

In this problem you're going to write a function **distance_from_angles_for_fmin** that you will pass to fmin in order to have **fmin** find the angles that bring the gripper location to the target points

TODO: Edit do_fmin to call func_for_fmin with the appropriate angles, etc


In [ ]:
def distance_from_angles_for_fmin(angles, arm_geometry, target):
    """ Compute the distance from the grasp point to the target
    @param angles as a numpy array, one angle for each joint
    @param arm_geometry - the same list of dictionaries we've been using for the arm compoments
    @param target - a tuple with the desired x,y location
    @return The distance between the target and the gripper grasp point"""

    # TODO: step 1, convert the numpy array into the format we've been using for the angles
    #  [a1, a2, a3, ... [ap, af, -af]]
    #  Set the angles of the fingers (af) to be zero
    #  If there are 3 links, then angles will have four values (the last angle is the wrist/palm angle, ap)

    angles_as_list = []
    ...

    # TODO: Call set_matrices_all_components with the arm_geometry and the angles you just created
    #    This sets the Matrix_pose's to the correct values for these angles
    ...

    # TODO - call get_gripper_location to get gx,gy (the gripper location with these angles)
    ...

    # TODO - return any one of the above distance values. This should be a single number
    return abs(gx - target[0]) + abs(gy - target[1])

In [ ]:
# Set the starting angles of the arm
angles_start = [np.pi/6.0, -np.pi/4, 1.5 * np.pi/4, [np.pi/3.0, -np.pi/8.0, np.pi/6.0]]
set_matrices_all_components(arm_geometry, angles_start)

# Get the gripper location
gx, gy = get_gripper_location(arm_geometry)

# Check that your function returns zero
#   This converts the start angles to a numpy array
angles_start_flat_list = [a for a in angles_start[0:-1]]
angles_start_flat_list.append(angles_start[-1][0])                          
angles_start_as_np_array = np.array(angles_start_flat_list)
dist = distance_from_angles_for_fmin(angles_start_as_np_array, arm_geometry, (gx, gy))

assert(np.isclose(dist, 0.0))

# Try some angles that should be further away
angles_not_start_as_np_array = angles_start_as_np_array * 0.5
dist_far_away = distance_from_angles_for_fmin(angles_not_start_as_np_array, arm_geometry, (gx, gy))

assert(dist_far_away > 0.0)

In [ ]:
grader.check("optimization_dist_func")

## Do the fmin call

Here you're going to set up the **fmin** call. As always, you can write this outside of the function call then put it in the function after it's working

- First, convert the starting angles to a numpy array (see previous question)
- Next, call fmin with the function distance_from_angles_for_fmin and a tuple that has the arm_geometry and the target in it


In [ ]:
def do_fmin(angles_start, arm_geometry, target):
    """ Set the angles/matrices of arm_geometry so they reach the target point
    @param - angles to start with
    @param - the arm_geometry list of dictionaries
    @param - the target as a tuple (x,y)
    @return the arm geometry with the angles set to the result of calling fmin
    """
    # TODO - convert the list of angles to a numpy array (see above)
    angles_as_nparray = np.array([0, 0])
    ...

    # TODO - now call fmin with your function, the starting angles, and the remaining paramters in args
    ...
    # TODO - set arm_geometry's Matrix_pose to have the ones for these joint angles
    #   You can either convert the angles back to a list or just call distance_from_angles_for_fmin
    ...
    return arm_geometry

In [ ]:
# You must start with these angles
angles_start = [np.pi/6.0, -np.pi/4, 1.5 * np.pi/4, [np.pi/3.0, -np.pi/8.0, np.pi/6.0]]
# The target
target = np.array([0.55, 1.15])

arm_geometry_optimized = do_fmin(angles_start, arm_geometry, target)

In [ ]:
assert(np.isclose(get_gripper_location(arm_geometry_optimized)[0], target[0], atol=0.01))
assert(np.isclose(get_gripper_location(arm_geometry_optimized)[1], target[1], atol=0.01))

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
plot_complete_arm(axs, arm_geometry_optimized)
axs.plot(target[0], target[1], '+g', markersize=20)

In [ ]:
grader.check("do_fmin")

# Generalization

If nothing has been "hardwired" in, this should just work - changing the geometry, the starting angles, the target point. However, if you've hardwired something in, it probably won't...

In [ ]:
# Create the arm geometry
base_size_param = (0.5, 0.25) # squished
link_sizes_param = [(0.3, 0.15), (0.2, 0.09), (0.1, 0.05), (0.075, 0.03)]
palm_width_param = 0.15
finger_size_param = (0.085, 0.015)


# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_longer = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)

# Set the angles of the arm
angles_start_longer = [-np.pi/4.0, -np.pi/4, 1.2 * np.pi/4, -1 * np.pi/8, [-np.pi/3.0, np.pi/6.0, -np.pi/6.0]]
set_matrices_all_components(arm_longer, angles_start_longer)

target_longer = np.array([0.3, -0.15])

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_longer)
axs.plot(target_longer[0], target_longer[1], '+g', markersize=20)

In [ ]:
# Do the optimization
arm_longer_optimized = do_fmin(angles_start_longer, arm_longer, target_longer)

In [ ]:
# Plot arm with target
fig, axs = plt.subplots(1, 1, figsize=(8, 8))
plot_complete_arm(axs, arm_longer_optimized)
axs.plot(target_longer[0], target_longer[1], '+g', markersize=20)

In [ ]:
assert(np.isclose(get_gripper_location(arm_longer_optimized)[0], target_longer[0], atol=0.01))
assert(np.isclose(get_gripper_location(arm_longer_optimized)[1], target_longer[1], atol=0.01))

In [ ]:
grader.check("generalization")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS TAF4 (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

### To submit

(Did you read me?)

- Submit this .ipynb file AND arm_routines.py. If you don't include arm_routines.py Gradescope cannot magically reach out to your computer and find your arm_routines.py file.
- We will supply matrix_routines.py for you (it won't break anything if you do include it)

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

Lots of people forget arm_routines.py. 

Make sure you remove all the print statements you put in that print out lots of stuff.